# MEOP dataset preprocessing
MEOP data can be found at: http://www.meop.net/database/format.html
MEOP stands for: "Marine Mammals Exploring the Oceans from Pole to Pole".
MEOP consists of ARGO like profiles, however, the instruments are fitted onto marine mammals.
The UK dataset of MEOP contains many profiles in the Weddel Sea region, the following analysis is of this dataset.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gsw
from netCDF4 import Dataset
import xarray as xr
import pdb
from IPython.display import Image
from scipy.interpolate import griddata
from scipy.io import netcdf

Make sure that all the paths to the netcdf files are given in a column in the file named "filenames.txt"

This can be done by 
1. navigating to the folder in the terminal and typing in: "ls -1 | grep .nc > filenames.txt"
2. copy and paste "filenames.txt" into the working folder of the Jupyter Notebook
3. append the full path name to the file list by:  cat filenames.txt | xargs -i echo "/path/to/ncARGO/{}" > filenames.txt


In [3]:
!ls ../ncARGO -1 | grep .nc

ct1-Blitzen-04_prof.nc
ct1-Comet-04_prof.nc
ct1-Dancer-04_prof.nc
ct1-Dasher-04_prof.nc
ct1-Donner-04_prof.nc
ct1-Vixen-04_prof.nc
ct27-W1-07_prof.nc
ct27-W2-07_prof.nc
ct27-W3-07_prof.nc
ct27-W5-07_prof.nc
ct27x-W4-07_prof.nc
ct40-848-08_prof.nc
ct40-853-08_prof.nc
ct40-857-08_prof.nc
ct40-861-08_prof.nc
ct40-Bernt-08_prof.nc
ct40-Bob-08_prof.nc
ct40-Boris-08_prof.nc
ct40-Lily-08_prof.nc
ct40-Rikk-08_prof.nc
ct40-Sheila-08_prof.nc
ct43-574-09_prof.nc
ct43-582-09_prof.nc
ct43-613-09_prof.nc
ct43-858-09_prof.nc
ct43-860-09_prof.nc
ct43-862-09_prof.nc
ct43-864-09_prof.nc
ct43-865-09_prof.nc
ct43-866-09_prof.nc
ct45-BradBingley-08_prof.nc
ct45-FannieMae-08_prof.nc
ct45-Fortis-08_prof.nc
ct45-FreddieMac-08_prof.nc
ct45-HBOS-08_prof.nc
ct45-Landsbanki-08_prof.nc
ct45-LehmanBros-08_prof.nc
ct45-NorthernRock-08_prof.nc
ct45-RBS-08_prof.nc
ct45-WashMutual-08_prof.nc
ct49-Adnams-09_prof.nc
ct49-Amstel-09_prof.nc
ct49-Brahma-09_prof.nc
ct49-Carling-09_prof.nc
ct49-Corona-09_prof.nc
ct49-Grolsch-

In [12]:
Blitzen = netcdf.netcdf_file("../ncARGO/ct1-Blitzen-04_prof.nc", 'r')

In [25]:
Blitzen.variables

OrderedDict([('DATA_TYPE',
              <scipy.io.netcdf.netcdf_variable at 0x7f76bab85c88>),
             ('FORMAT_VERSION',
              <scipy.io.netcdf.netcdf_variable at 0x7f76bab85c50>),
             ('HANDBOOK_VERSION',
              <scipy.io.netcdf.netcdf_variable at 0x7f76bab85d30>),
             ('REFERENCE_DATE_TIME',
              <scipy.io.netcdf.netcdf_variable at 0x7f76bab85dd8>),
             ('DATE_CREATION',
              <scipy.io.netcdf.netcdf_variable at 0x7f76bab85e48>),
             ('DATE_UPDATE',
              <scipy.io.netcdf.netcdf_variable at 0x7f76bab85ef0>),
             ('PLATFORM_NUMBER',
              <scipy.io.netcdf.netcdf_variable at 0x7f76bab85f28>),
             ('PROJECT_NAME',
              <scipy.io.netcdf.netcdf_variable at 0x7f76bab85f98>),
             ('PI_NAME', <scipy.io.netcdf.netcdf_variable at 0x7f76ba9aa080>),
             ('STATION_PARAMETERS',
              <scipy.io.netcdf.netcdf_variable at 0x7f76ba9aa128>),
             ('CYCLE

In [28]:
temp_error = Blitzen.variables['TEMP_ADJUSTED_ERROR'][:]
sal_error = Blitzen.variables['PSAL_ADJUSTED_ERROR'][:]
pres_error = Blitzen.variables['PRES_ADJUSTED_ERROR'][:]

In [34]:
np.max(temp_error[:][(temp_error[:] < 9e4)])

0.2

In [36]:
np.max(sal_error[:][sal_error < 9e4])

0.2

In [43]:
np.unique(pres_error)

array([ 99999.], dtype=float32)

In [1]:
!cat filenamesMerged.txt

../../AUSTRALIA/ncARGO/awru1-A-06_prof.nc
../../AUSTRALIA/ncARGO/awru1-B-06_prof.nc
../../AUSTRALIA/ncARGO/awru1-C-06_prof.nc
../../AUSTRALIA/ncARGO/ct100-251-13_prof.nc
../../AUSTRALIA/ncARGO/ct100-263-13_prof.nc
../../AUSTRALIA/ncARGO/ct100-264-13_prof.nc
../../AUSTRALIA/ncARGO/ct100-298-13_prof.nc
../../AUSTRALIA/ncARGO/ct100-301-13_prof.nc
../../AUSTRALIA/ncARGO/ct108-297-13_prof.nc
../../AUSTRALIA/ncARGO/ct108-300-13_prof.nc
../../AUSTRALIA/ncARGO/ct108-554-13_prof.nc
../../AUSTRALIA/ncARGO/ct108-555-13_prof.nc
../../AUSTRALIA/ncARGO/ct108-564-13_prof.nc
../../AUSTRALIA/ncARGO/ct108-569-13_prof.nc
../../AUSTRALIA/ncARGO/ct108-622-13_prof.nc
../../AUSTRALIA/ncARGO/ct108-632-13_prof.nc
../../AUSTRALIA/ncARGO/ct108-663-13_prof.nc
../../AUSTRALIA/ncARGO/ct109-031-14_prof.nc
../../AUSTRALIA/ncARGO/ct109-032-14_prof.nc
../../AUSTRALIA/ncARGO/ct109-085-14_prof.nc
../../AUSTRALIA/ncARGO/ct109-184-14_prof.nc
../../AUSTRALIA/ncARGO/ct109-186-14_prof.nc
../../AUSTRALIA/ncARGO/ct109-187-14_pr

In [5]:
filenamesMerged = pd.read_csv("filenamesMerged.txt", header=None)

In [6]:
#func to read in all the files in the filenames
def read_all_nc(filenames):
    data_array = [i for i in range(len(filenames))]
    i=0
    for i in range(len(filenames)):
        data_array[i] = xr.open_dataset(filenames[0][i])
    return data_array      

In [7]:
# func to create a merged pandas dataframe of all the netcdf files

def create_merged_df(arr_data):
    for i in range((len(arr_data))):
        
        nlev = len(arr_data[i]['PRES_ADJUSTED'][0])
        nprof = len(arr_data[i]['PRES_ADJUSTED'])
        if(i == 0):
            NPROF = np.arange(nprof)
            NPROF = np.array([[NPROF[j]] * nlev for j in range(len(NPROF)) ])
        else:
            last_end = NPROF.flatten()[-1] + 1
            NPROF = np.arange(last_end, last_end+nprof )
            NPROF = np.array([[NPROF[j]] * nlev for j in range(len(NPROF)) ])
        
        ind = np.arange(nprof*nlev)
        lat = np.array([[arr_data[i]["LATITUDE"][j].values]* nlev for j in range(nprof) ])
        lon = np.array([[arr_data[i]["LONGITUDE"][j].values]* nlev for j in range(nprof) ])
        posqc = np.array([[arr_data[i]["POSITION_QC"][j].values]* nlev for j in range(nprof) ])
        juld = np.array([[arr_data[i]["JULD"][j].values]* nlev for j in range(nprof) ])
        
        if(i == 0):    
            df = {'PLATFORM_NUMBER': pd.Series([str(arr_data[i]["PLATFORM_NUMBER"][0].values)]*len(ind), index=ind), 
                  'PROFILE_NUMBER' : pd.Series(NPROF.flatten(), index=ind ),
                  'TEMP_ADJUSTED': pd.Series(arr_data[i]["TEMP_ADJUSTED"].values.flatten(), index=ind), 
                  'PSAL_ADJUSTED': pd.Series(arr_data[i]["PSAL_ADJUSTED"].values.flatten(), index=ind), 
                  'PRES_ADJUSTED': pd.Series(arr_data[i]["PRES_ADJUSTED"].values.flatten(), index=ind),  
                  'TEMP_ADJUSTED_QC': pd.Series(arr_data[i]["TEMP_ADJUSTED_QC"].values.flatten(), index=ind), 
                  'PRES_ADJUSTED_QC': pd.Series(arr_data[i]["PRES_ADJUSTED_QC"].values.flatten(), index=ind), 
                  'PSAL_ADJUSTED_QC': pd.Series(arr_data[i]["PSAL_ADJUSTED_QC"].values.flatten(), index=ind),  
                  'JULD': pd.Series(juld.flatten(), index=ind),  
                  'LATITUDE': pd.Series(lat.flatten(), index=ind),  
                  'LONGITUDE': pd.Series(lon.flatten(), index=ind),  
                  'POSITION_QC': pd.Series(posqc.flatten(), index=ind) }
            df = pd.DataFrame(df)
        else:
            df_i = {'PLATFORM_NUMBER': pd.Series([str(arr_data[i]["PLATFORM_NUMBER"][0].values)]*len(ind), index=ind), 
                    'PROFILE_NUMBER' : pd.Series(NPROF.flatten(), index=ind ),
                    'TEMP_ADJUSTED': pd.Series(arr_data[i]["TEMP_ADJUSTED"].values.flatten(), index=ind), 
                    'PSAL_ADJUSTED': pd.Series(arr_data[i]["PSAL_ADJUSTED"].values.flatten(), index=ind), 
                    'PRES_ADJUSTED': pd.Series(arr_data[i]["PRES_ADJUSTED"].values.flatten(), index=ind),  
                    'TEMP_ADJUSTED_QC': pd.Series(arr_data[i]["TEMP_ADJUSTED_QC"].values.flatten(), index=ind), 
                    'PRES_ADJUSTED_QC': pd.Series(arr_data[i]["PRES_ADJUSTED_QC"].values.flatten(), index=ind), 
                    'PSAL_ADJUSTED_QC': pd.Series(arr_data[i]["PSAL_ADJUSTED_QC"].values.flatten(), index=ind),  
                    'JULD': pd.Series(juld.flatten(), index=ind),  
                    'LATITUDE': pd.Series(lat.flatten(), index=ind),  
                    'LONGITUDE': pd.Series(lon.flatten(), index=ind),  
                    'POSITION_QC': pd.Series(posqc.flatten(), index=ind) }
            df_i = pd.DataFrame(df_i)
            
            #pdb.set_trace()
            df = df.append(df_i, ignore_index=True)
            
    return df

In [8]:
alldata = read_all_nc(filenamesMerged[0:5])
dfm = create_merged_df(alldata)
print(len(alldata))

5


In [10]:
alldata[0].variables

Frozen(OrderedDict([('DATA_TYPE', <xarray.Variable ()>
b'Argo profile    '
Attributes:
    comment: Data type), ('FORMAT_VERSION', <xarray.Variable ()>
b'3.0 '
Attributes:
    comment: File format version), ('HANDBOOK_VERSION', <xarray.Variable ()>
b'3.0 '
Attributes:
    comment: Data handbook version), ('REFERENCE_DATE_TIME', <xarray.Variable ()>
b'19500101000000'
Attributes:
    comment: Date of reference for Julian days
    conventions: YYYYMMDDHHMISS), ('DATE_CREATION', <xarray.Variable ()>
b'20160710194158'
Attributes:
    comment: Date of file creation
    conventions: YYYYMMDDHHMISS), ('DATE_UPDATE', <xarray.Variable ()>
b'20160710194158'
Attributes:
    long_name: Date of update of this file
    conventions: YYYYMMDDHHMISS), ('PLATFORM_NUMBER', <xarray.Variable (N_PROF: 893)>
array([b'00012048', b'00012048', b'00012048', b'00012048', b'00012048',
       b'00012048', b'00012048', b'00012048', b'00012048', b'00012048',
       b'00012048', b'00012048', b'00012048', b'00012048', b

In [7]:
dfm.head()

,JULD,LATITUDE,LONGITUDE,PLATFORM_NUMBER,POSITION_QC,PRES_ADJUSTED,PRES_ADJUSTED_QC,PROFILE_NUMBER,PSAL_ADJUSTED,PSAL_ADJUSTED_QC,TEMP_ADJUSTED,TEMP_ADJUSTED_QC
0,2007-03-03 03:24:00.028799999,-68.5139,78.3736,b'00012048',b'1',6.0,b'1',0,NaN,b'9',7.450581e-10,b'1'
1,2007-03-03 03:24:00.028799999,-68.5139,78.3736,b'00012048',b'1',8.0,b'1',0,NaN,b'9',-6.640000e-02,b'1'
2,2007-03-03 03:24:00.028799999,-68.5139,78.3736,b'00012048',b'1',14.0,b'1',0,NaN,b'9',-3.985000e-01,b'1'
3,2007-03-03 03:24:00.028799999,-68.5139,78.3736,b'00012048',b'1',18.0,b'1',0,NaN,b'9',-4.852000e-01,b'1'
4,2007-03-03 03:24:00.028799999,-68.5139,78.3736,b'00012048',b'1',22.0,b'1',0,NaN,b'9',-6.734000e-01,b'1'


In [8]:
dfm['TEMP_ADJUSTED_QC'] = dfm['TEMP_ADJUSTED_QC'].str.decode("utf-8")
dfm['PRES_ADJUSTED_QC'] = dfm['PRES_ADJUSTED_QC'].str.decode("utf-8")
dfm['PSAL_ADJUSTED_QC'] = dfm['PSAL_ADJUSTED_QC'].str.decode("utf-8")

In [9]:
dfm["JULD"] = pd.to_datetime(dfm['JULD'],infer_datetime_format=True)

In [10]:
dfm["DEPTH"] = gsw.z_from_p(dfm["PRES_ADJUSTED"], dfm["LATITUDE"])

In [11]:
SA = gsw.SA_from_SP(dfm['PSAL_ADJUSTED'],dfm['PRES_ADJUSTED'],dfm['LONGITUDE'],dfm['LATITUDE'])
CT = gsw.CT_from_t(SA, dfm['TEMP_ADJUSTED'], dfm['PRES_ADJUSTED'])
dfm['DENSITY_INSITU'] = gsw.density.rho(SA, CT, dfm['PRES_ADJUSTED'])
dfm['POT_DENSITY'] = gsw.density.sigma0(SA, CT)
dfm['CTEMP'] = CT
del(SA, CT)

In [12]:
mask_notbad_temp = ~(dfm['TEMP_ADJUSTED_QC'].str.contains('4'))
mask_notbad_sal = ~(dfm['PSAL_ADJUSTED_QC'].str.contains('4'))
mask_notbad_pres = ~(dfm['PRES_ADJUSTED_QC'].str.contains('4'))
mask_below60S = (dfm['LATITUDE'] < -60)
mask_all = (mask_notbad_pres & mask_notbad_sal & mask_notbad_temp & mask_below60S)

In [13]:
def compute_density_gradients(df):
    sigma0 = df['POT_DENSITY'].as_matrix()
    z = df['DEPTH'].as_matrix()
    
    dinv = np.zeros(len(sigma0))
    dz = np.zeros(len(z))
    
    dinv[1:] = sigma0[1:] - sigma0[:-1]
    dinv[0] = sigma0[1] - sigma0[0]
    
    dz[1:] = z[1:] - z[:-1]
    dz[0] = z[1] - z[0]
    
    df['DENSITY_GRADIENT'] = dinv/dz
    return df

In [14]:
dfm.loc[:,'DENSITY_GRADIENT'] = np.nan
mask_SD = ~dfm.loc[:,'POT_DENSITY'].isnull()
dfm.loc[mask_all & mask_SD] = dfm.loc[mask_all & mask_SD].groupby('PROFILE_NUMBER').\
                                            apply(compute_density_gradients)

In [15]:
bathy = netcdf.netcdf_file('/media/data/Datasets/Bathymetry/GEBCO_2014_2D.nc', 'r')
lons = bathy.variables['lon'][:]
lats = bathy.variables['lat'][:]
dfm['ECHODEPTH'] = np.nan

def assign_echodepth(df):
    ind = df.index
    lon = df.loc[ind[0], 'LONGITUDE']
    lat = df.loc[ind[0],'LATITUDE']
    ind_lon = np.argmin(np.abs(lons - lon))
    ind_lat = np.argmin(np.abs(lats - lat))
    df.loc[:,'ECHODEPTH'] = bathy.variables['elevation'][ind_lat, ind_lon]

    return df
    
dfm.loc[mask_all] = dfm.loc[mask_all].groupby(['LATITUDE', 'LONGITUDE']).apply(assign_echodepth)

In [32]:
def play_beep():
    import os
    duration = 1  # second
    freq = 440  # Hz
    os.system('play --no-show-progress --null --channels 1 synth %s sine %f' % (duration, freq))
play_beep()

In [36]:
len(dfm.loc[mask_all])

2884471

In [37]:
dfm.loc[mask_all].to_csv("dfmg.csv")